<a href="https://colab.research.google.com/github/pramod4A3/Data/blob/main/fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c fake-news

 88% 41.0M/46.5M [00:00<00:00, 69.1MB/s]
100% 46.5M/46.5M [00:00<00:00, 72.3MB/s]


In [ ]:
!unzip fake-news.zip

Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


**Import libraries**

In [ ]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
rr=pd.read_csv("/content/train.csv")
print(rr.shape)
rr.describe()

(20800, 5)


,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [ ]:
rr.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
rr=rr.drop("id",axis=1)

In [ ]:
rr.tail()

,title,author,text,label
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1
20799,What Keeps the F-35 Alive,David Swanson,"David Swanson is an author, activist, journa...",1


In [ ]:
rr.isnull().sum()

title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
df=rr.dropna()

In [ ]:
df.isnull().sum()

title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
df.shape

(18285, 4)

In [ ]:
col_obj=df.select_dtypes(include=["object"])
col_obj.columns

Index(['title', 'author', 'text'], dtype='object')

In [ ]:
mess=df.copy()

In [ ]:
mess["content"]=mess["title"]+" "+mess["author"]+" "+mess["text"]
mess

,title,author,text,label,content
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You Fired Consortiumne...
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...
...,...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Rapper T.I.: Trump a ’Poster Child For White S...
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"N.F.L. Playoffs: Schedule, Matchups and Odds -..."
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Macy’s Is Said to Receive Takeover Approach by...
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
x=mess["content"]
y=mess['label']

In [ ]:
from nltk.stem.porter import PorterStemmer
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub(r'[0-9]+|[^\w\s]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

mess['content'] = mess['content'].apply(stemming)

In [ ]:
import concurrent.futures

def preprocess_text(text):
    # Remove numbers and non-alphanumeric characters, convert to lowercase
    text = re.sub(r'[0-9]+|[^\w\s]', ' ', text).lower()

    # Split the text into words
    words = text.split()

    # Perform stemming and remove stopwords
    stemmed_words = [port_stem.stem(word) for word in words if word not in stopwords.words('english')]


    return ' '.join(stemmed_words)
# Define a function to preprocess a DataFrame column in parallel
def preprocess_column(column):
    with concurrent.futures.ProcessPoolExecutor() as executor:
        processed_texts = list(executor.map(preprocess_text, column))
    return processed_texts

mess["content"]=preprocess_column(mess['content'])

In [ ]:
vec=TfidfVectorizer()
x=vec.fit_transform(x)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=13)

In [ ]:
x_train.shape

(14628, 161513)

In [ ]:
y_train


2874     1
6155     0
17579    1
6818     0
13388    1
        ..
8440     0
1204     1
989      0
1665     0
8767     0
Name: label, Length: 14628, dtype: int64

In [ ]:
print(x_test)

  (0, 10739)	0.07942070965949005
  (0, 59596)	0.030437308638754986
  (0, 59881)	0.027821211747733218
  (0, 55165)	0.03478922095458042
  (0, 20097)	0.03156647023191453
  (0, 78081)	0.02933252047989189
  (0, 8140)	0.031043962470399103
  (0, 114388)	0.029553515533246224
  (0, 118284)	0.029553515533246224
  (0, 22184)	0.03029873900182266
  (0, 35426)	0.03371682989652285
  (0, 90197)	0.023830779187793
  (0, 19713)	0.026090259396459627
  (0, 5840)	0.023612732142370182
  (0, 30257)	0.0600698868463003
  (0, 103572)	0.02238411520904075
  (0, 131980)	0.028923438068330153
  (0, 8138)	0.025994080539687516
  (0, 31215)	0.4028508733463814
  (0, 99758)	0.03521987287698727
  (0, 130200)	0.021317775096307157
  (0, 4607)	0.04651416144917821
  (0, 43176)	0.02558726199571058
  (0, 110306)	0.03072939092422952
  (0, 34423)	0.057654518423116086
  :	:
  (3656, 94005)	0.12275476637950306
  (3656, 131795)	0.09904457462917078
  (3656, 51900)	0.02231652059768791
  (3656, 110143)	0.0319332344955994
  (3656, 15315)

In [ ]:
model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [ ]:
# accuracy score on the training data
x_train_pred = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_pred, y_train)

In [ ]:
training_data_accuracy

0.978192507519825

In [ ]:
from sklearn.metrics import accuracy_score

# Assuming you have already trained a machine learning model and made predictions on the training data
# Replace `your_model` with the actual variable name of your model
# Replace `x_train_prediction` and `y_train` with the actual variable names for predictions and labels

model.fit(x_train, y_train)  # Assuming you have already trained your model

# Make predictions on the training data
x_train_prediction =model.predict(x_train)

# Calculate accuracy on the training data
training_data_accuracy = accuracy_score(y_train, x_train_prediction)

# Print the accuracy
print("Training Data Accuracy:", training_data_accuracy)


Training Data Accuracy: 0.978192507519825
